Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [15]:
import sys
#!conda install --yes --prefix {sys.prefix} scikit-learn

Solving environment: done

## Package Plan ##

  environment location: /home/mortennp/anaconda3/envs/AdvML3_6

  added / updated specs: 
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1a             |       h7b6447c_0         5.0 MB
    libgcc-7.2.0               |       h69d50b8_2         304 KB
    pyqt-5.6.0                 |           py36_2         5.4 MB
    scikit-learn-0.20.1        |   py36h4989274_0         5.7 MB
    mkl-2018.0.3               |                1       198.7 MB
    sqlite-3.25.3              |       h7b6447c_0         1.9 MB
    cryptography-2.4.1         |   py36h1ba5d50_0         619 KB
    qt-5.6.3                   |       h8bf5577_3        45.7 MB
    grpcio-1.16.1              |   py36hf8bcb03_1         1.1 MB
    ------------------------------------------------------------
                                           Total

In [16]:
import pandas as pd
import numpy as np
from itertools import product
from grader import Grader
#import sklearn as skl

# Read data

In [2]:
sales = pd.read_csv('../readonly/final_project_data/sales_train.csv.gz')

# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [3]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

/home/mortennp/anaconda3/envs/AdvML3_6/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [4]:
all_data.target.describe()

count    1.091385e+07
mean     3.342731e-01
std      3.417243e+00
min     -2.200000e+01
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.253000e+03
Name: target, dtype: float64

# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

#### Method 1

In [5]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862128


In [6]:
all_data.tail()

,shop_id,item_id,date_block_num,target,item_target_enc
10768834,59,22162,33,0.0,1.556793
10769024,59,22163,33,0.0,0.581395
10769690,59,22164,33,0.0,1.235589
10771216,59,22166,33,0.0,0.295918
10770511,59,22167,33,0.0,1.081081


#### Method 2

In [7]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862128


In [8]:
all_data.tail()

,shop_id,item_id,date_block_num,target,item_target_enc
10768834,59,22162,33,0.0,1.556793
10769024,59,22163,33,0.0,0.581395
10769690,59,22164,33,0.0,1.235589
10771216,59,22166,33,0.0,0.295918
10770511,59,22167,33,0.0,1.081081


See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [9]:
grader = Grader()

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [32]:
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)

,shop_id,item_id,date_block_num,target
10913845,21,7635,33,0.0
10913846,21,7638,33,0.0
10913847,21,7640,33,0.0
10913848,21,7632,33,0.0
10913849,21,7440,33,0.0


In [38]:
# YOUR CODE GOES HERE
from sklearn.model_selection import KFold
y = all_data['target'].values
for tr_idx, vali_idx in KFold(5, shuffle=False).split(y):
    X_tr, X_vali = all_data.iloc[tr_idx], all_data.iloc[vali_idx]
    all_data.loc[vali_idx, 'item_target_enc'] = X_vali['item_id'].map(X_tr.groupby('item_id').target.mean()) 
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

In [39]:
all_data.tail()

,shop_id,item_id,date_block_num,target,item_target_enc
10913845,21,7635,33,0.0,0.334300
10913846,21,7638,33,0.0,0.020408
10913847,21,7640,33,0.0,0.334300
10913848,21,7632,33,0.0,0.020202
10913849,21,7440,33,0.0,0.334300


In [40]:
encoded_feature = all_data['item_target_enc'].values

# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('KFold_scheme', corr)

0.4166330182245754
Current answer for task KFold_scheme is: 0.4166330182245754


# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [107]:
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)

In [108]:
# YOUR CODE GOES HERE
all_data['item_target_enc'] = (all_data.groupby('item_id')['target'].transform('sum') - all_data['target']) / (all_data.groupby('item_id')['target'].transform('count') - 1)
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

encoded_feature = all_data['item_target_enc'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Leave-one-out_scheme', corr)

0.48038483112911484
Current answer for task Leave-one-out_scheme is: 0.48038483112911484


# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [109]:
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)

In [110]:
# YOUR CODE GOES HERE
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').agg({'target':['mean', 'count']})
item_id_target_mean.columns = item_id_target_mean.columns.map('_'.join)
#item_id_target_mean.reset_index()
item_id_target_mean.tail()

,target_mean,target_count
item_id,,
22165,0.021277,94
22166,0.295918,1176
22167,1.081081,1221
22168,0.032967,182
22169,0.020833,48


In [111]:
alpha = 100
globalmean = 0.3343 
item_id_target_mean = item_id_target_mean.assign(item_target_enc=
                                                 ((item_id_target_mean.target_mean * item_id_target_mean.target_count) +
                                                 globalmean * alpha) /
                                                 (item_id_target_mean.target_count + alpha))
                                                 
item_id_target_mean.tail()
item_id_target_mean = item_id_target_mean['item_target_enc']

In [112]:
# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

In [113]:
encoded_feature = all_data['item_target_enc'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Smoothing_scheme', corr)

0.4818198797095273
Current answer for task Smoothing_scheme is: 0.4818198797095273


# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [129]:
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)

In [130]:
# YOUR CODE GOES HERE
cumsum = all_data.groupby('item_id')['target'].cumsum() - all_data['target']
cumcnt = all_data.groupby('item_id').cumcount()

In [134]:
all_data['item_target_enc'] = cumsum / cumcnt
all_data.query('item_id == 0').tail()

,shop_id,item_id,date_block_num,target,item_target_enc
7425459,57,0,20,0.0,0.022222
7431530,58,0,20,0.0,0.021739
7437601,59,0,20,0.0,0.021277
7443672,55,0,20,0.0,0.020833
7449743,56,0,20,0.0,0.020408


In [131]:
# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

encoded_feature = all_data['item_target_enc'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Expanding_mean_scheme', corr)

0.5037485458166915
Current answer for task Expanding_mean_scheme is: 0.5037485458166915


## Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. Note: Token expires 30 minutes after generation.

In [119]:
STUDENT_EMAIL = 'mortennp@hotmail.com'
STUDENT_TOKEN = 'kdWQLm6qDMzmsC9c'
grader.status()

You want to submit these numbers:
Task KFold_scheme: 0.4166330182245754
Task Leave-one-out_scheme: 0.48038483112911484
Task Smoothing_scheme: 0.4818198797095273
Task Expanding_mean_scheme: 0.5037485458166915


In [120]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
